### Test of CV functionality. 

Most storage related tests are unit tests now. So look in `testcollectivevariable.py` please.

In [1]:
from __future__ import print_function
import openpathsampling as paths
import numpy as np
import openpathsampling.engines.openmm as peng

In [2]:
# force numpy print options for test comparison
np.set_printoptions(precision=6, formatter={'float_kind': lambda x: "{:.6f}".format(x)})

Load first frame from test pdb.

In [3]:
template = peng.snapshot_from_pdb('../resources/AD_initial_frame.pdb')

Create a simple CV without an underlying function.

In [4]:
cv0 = paths.CollectiveVariable('func0')

Create a complicated function.

In [5]:
center = 1
def dist(snapshot, center, np):
    return np.sum(snapshot.coordinates._value[0]) - center

Create collective variable from this function. Note that you have to specify `center` and `np` to make this work

In [6]:
cv1 = paths.FunctionCV('func1', dist, center=center, np=np, cv_time_reversible=False).with_diskcache()
cv2 = paths.FunctionCV('func2', dist, center=center, np=np, cv_wrap_numpy_array=True).with_diskcache()
cv3 = paths.FunctionCV('func3', dist, center=center, np=np, cv_wrap_numpy_array=True, cv_time_reversible=True).with_diskcache()

Create storage to test save and load.

In [7]:
# NBVAL_IGNORE_OUTPUT
storage = paths.Storage('can_be_deleted.nc', mode='w')
print(storage.snapshots.save(template))

306465958118793424807025199580056649754


Save CV

In [8]:
# NBVAL_IGNORE_OUTPUT
print(storage.save([cv0, cv1, cv2, cv3]))

[(store.attributes[PseudoAttribute] : 4 object(s), 20, 306465958118793424807025199580056649756), (store.attributes[PseudoAttribute] : 4 object(s), 20, 306465958118793424807025199580056649758), (store.attributes[PseudoAttribute] : 4 object(s), 20, 306465958118793424807025199580056649760), (store.attributes[PseudoAttribute] : 4 object(s), 20, 306465958118793424807025199580056649762)]


In [9]:
# NBVAL_IGNORE_OUTPUT
print(storage.cvs.index)

{306465958118793424807025199580056649756: 0, 306465958118793424807025199580056649758: 1, 306465958118793424807025199580056649760: 2, 306465958118793424807025199580056649762: 3}


Set the CV value for the storage.template

In [10]:
cv0[template] = 10.0

and create a storage for it

In [11]:
storage.cvs.add_diskcache(cv0, allow_incomplete=True)

And we should have a float store

In [12]:
storage.cvs.sync(cv0)

In [13]:
assert(cv0._store_dict.value_store.vars['value'][:] == [10.0])

Test function for reversed template.

In [14]:
dd = dist(template.reversed, center, np)
print("{:.8f}".format(dd))

3.39210033


Create another CV. This time using the `from_template` function.

In [15]:
print(cv0([template, template]))
print(cv1([template, template]))

[10.0, 10.0]
[np.float32(3.3921003), np.float32(3.3921003)]


In [16]:
# NBVAL_SKIP
print(type(cv0([template, template])))
print(type(cv0([template, template])[0]))
print(type(cv1([template, template])))
print(type(cv1([template, template])[0]))

<class 'list'>
<class 'float'>
<class 'list'>
<class 'numpy.float32'>


In [17]:
# NBVAL_SKIP
print(storage.cvs.variables['json'][:])

['{"_cls":"CollectiveVariable","_dict":{"name":"func0","cv_time_reversible":false}}'
 '{"_cls":"FunctionCV","_dict":{"name":"func1","cv_time_reversible":false,"f":{"__callable_name__":"dist","_dilled":"gASVpgEAAAAAAACMCmRpbGwuX2RpbGyUjBBfY3JlYXRlX2Z1bmN0aW9ulJOUKGgAjAxfY3JlYXRl\\nX2NvZGWUk5QoQwICAZRLA0sASwBLA0sESwNDUpcAfAKgAAAAAAAAAAAAAAAAAAAAAAAAAAAAfABq\\nAQAAAAAAAAAAagIAAAAAAAAAAGQBGQAAAAAAAAAAAKYBAACrAQAAAAAAAAAAfAF6CgAAUwCUTksA\\nhpSMA3N1bZSMC2Nvb3JkaW5hdGVzlIwGX3ZhbHVllIeUjAhzbmFwc2hvdJSMBmNlbnRlcpSMAm5w\\nlIeUjE4vdmFyL2ZvbGRlcnMvdmovMjhjMTA3NDk2c3E1ejR5MTByano1Z2RtMDAwMGduL1QvaXB5\\na2VybmVsXzk2Mjk5LzQxODQ5NTE0NDUucHmUjARkaXN0lGgRSwJDJIAA2AsNjzaKNpAo1BIm1BIt\\nqGHUEjDRCzHUCzGwRtELOtAEOpRDAJQpKXSUUpR9lIwIX19uYW1lX1+UjAhfX21haW5fX5RzaBFO\\nTnSUUpR9lH2UjA9fX2Fubm90YXRpb25zX1+UfZRzhpRiLg==\\n"},"cv_requires_lists":false,"cv_wrap_numpy_array":false,"cv_scalarize_numpy_singletons":false,"kwargs":{"center":1,"np":{"_import":"numpy"}}}}'
 '{"_cls":"FunctionCV","_dict":{"name":"func2","c

In [18]:
cv0j = storage.cvs.vars['json'][0]
cv1j = storage.cvs.vars['json'][1]
cv2j = storage.cvs.vars['json'][2]
cv3j = storage.cvs.vars['json'][3]

In [19]:
res = cv0j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv1j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv2j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv3j([template, template, template])
assert(res[0] == res[1] == res[2])

#### PseudoAttributes

In [20]:
t = paths.Trajectory([template] * 4)

In [21]:
# NBVAL_IGNORE_OUTPUT
print(storage.save(t))

(store.trajectories[Trajectory] : 1 object(s), 0, 306465958118793424807025199580056649838)


In [22]:
def ff(t, cv3):
    return max(cv3(t))

In [23]:
a = paths.netcdfplus.FunctionPseudoAttribute('max_cv', paths.Trajectory, ff, cv3=cv3)

In [24]:
# NBVAL_IGNORE_OUTPUT
print(storage.attributes.save(a))

306465958118793424807025199580056649840


In [25]:
print("{:.8f}".format(a(t)))

3.39210033


In [26]:
p = paths.netcdfplus.LoaderProxy(storage.trajectories, t.__uuid__)

In [27]:
print("{:.8f}".format(a(p)))

3.39210033


In [28]:
print(storage.trajectories.add_attribute(paths.netcdfplus.ValueStore, a, t, allow_incomplete=False))

store.trajectories_max_cv[None/ANY] : (not created)


In [29]:
ats = storage.trajectories.attribute_list[a]

In [30]:
print(ats.vars['value'][:])

[3.392100]


In [31]:
print(a._store_dict.value_store.vars['value'][:])

[3.392100]


In [32]:
storage.trajectories.sync_attribute(a)

In [33]:
storage.attributes.has_cache(a)

True

In [34]:
storage.attributes.sync(a)

In [35]:
storage.close()

In [36]:
storage = paths.Storage('can_be_deleted.nc', mode='r')

In [37]:
# NBVAL_SKIP
storage.attributes[4]

In [38]:
storage.close()